## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [19]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AAPL", "AMD", "SMCI", "PCT", "NVDA"],
    interval="1d",
    start_date="2020-01-01",
    end_date="2024-01-01",
)
toolbox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toolbox(symbol=['AAPL', 'AMD', 'SMCI', 'PCT', 'NVDA'], interval=1d, start_date=2020-01-01, end_date=2024-01-01, provider=yfinance)

## Run Mandelbrot Channel Calculation

In [21]:
result = toolbox.technical.mandelbrot_channel(historical=True, window="1m")
result
result.to_polars()

In [15]:
result.to_polars().write_parquet("../tests/test_data/mandelbrot_channel_historical.parquet")